In [11]:
import findspark
findspark.init("/opt/homebrew/opt/apache-spark/libexec/")


In [12]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [13]:
#pysparkに必要なライブラリを読み込む

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sql("show tables").show() 

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|jinko_avg|      false|
+---------+---------+-----------+



In [14]:
df = spark.read.option("multiLine","true").option("encoding","SJIS").csv("/Users/isomasaki/pyspark_batch/dataset/jinko.csv",header=True,sep=",",inferSchema=False)
df.count()

982

In [15]:
df.show(truncate=False)


+--------------+----------+----+----------+----------+----+------------+----------+----------+
|都道府県コード|都道府県名|元号|和暦（年）|西暦（年）|注  |人口（総数）|人口（男）|人口（女）|
+--------------+----------+----+----------+----------+----+------------+----------+----------+
|00            |全国      |大正|9         |1920      |null|55963053    |28044185  |27918868  |
|01            |北海道    |大正|9         |1920      |null|2359183     |1244322   |1114861   |
|02            |青森県    |大正|9         |1920      |null|756454      |381293    |375161    |
|03            |岩手県    |大正|9         |1920      |null|845540      |421069    |424471    |
|04            |宮城県    |大正|9         |1920      |null|961768      |485309    |476459    |
|05            |秋田県    |大正|9         |1920      |null|898537      |453682    |444855    |
|06            |山形県    |大正|9         |1920      |null|968925      |478328    |490597    |
|07            |福島県    |大正|9         |1920      |null|1362750     |673525    |689225    |
|08            |茨城県    |大正|9       

In [16]:
#日本語のカラム名をローマ字へ変換
from pyspark.sql.types import StructType,StructField,StringType
from pyspark.sql.functions import col

struct = StructType([
    StructField("code",StringType(),False),
    StructField("kenmei",StringType(),False),
    StructField("gengo",StringType(),False),
    StructField("wareki",StringType(),False),
    StructField("seireki",StringType(),False),
    StructField("chu",StringType(),False),
    StructField("sokei",StringType(),False),
    StructField("jinko_male",StringType(),False),
    StructField("jinko_female",StringType(),False)
])

df = spark.read.option("multiLine","true").option("encoding","SJIS") \
.csv("/Users/isomasaki/pyspark_batch/dataset/jinko.csv",header=False,sep=",",inferSchema=False,schema=struct)
df.show()

+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|          code|    kenmei|gengo|    wareki|   seireki| chu|       sokei|jinko_male|jinko_female|
+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|都道府県コード|都道府県名| 元号|和暦（年）|西暦（年）|  注|人口（総数）|人口（男）|  人口（女）|
|            00|      全国| 大正|         9|      1920|null|    55963053|  28044185|    27918868|
|            01|    北海道| 大正|         9|      1920|null|     2359183|   1244322|     1114861|
|            02|    青森県| 大正|         9|      1920|null|      756454|    381293|      375161|
|            03|    岩手県| 大正|         9|      1920|null|      845540|    421069|      424471|
|            04|    宮城県| 大正|         9|      1920|null|      961768|    485309|      476459|
|            05|    秋田県| 大正|         9|      1920|null|      898537|    453682|      444855|
|            06|    山形県| 大正|         9|      1920|null|      968925|    478328|      49059

In [17]:
df.where(df.gengo == "平成").show()


+----+----------------------+-----+------+-------+----+---------+----------+------------+
|code|                kenmei|gengo|wareki|seireki| chu|    sokei|jinko_male|jinko_female|
+----+----------------------+-----+------+-------+----+---------+----------+------------+
|  00|                  全国| 平成|     2|   1990|null|123611167|  60696724|    62914443|
|  0A|          人口集中地区| 平成|     2|   1990|null| 78152452|  38564229|    39588223|
|  0B|人口集中地区以外の地区| 平成|     2|   1990|null| 45458715|  22132495|    23326220|
|  01|                北海道| 平成|     2|   1990|null|  5643647|   2722988|     2920659|
|  02|                青森県| 平成|     2|   1990|null|  1482873|    704758|      778115|
|  03|                岩手県| 平成|     2|   1990|null|  1416928|    680197|      736731|
|  04|                宮城県| 平成|     2|   1990|null|  2248558|   1105103|     1143455|
|  05|                秋田県| 平成|     2|   1990|null|  1227478|    584678|      642800|
|  06|                山形県| 平成|     2|   1990|null|  1258390|

In [18]:
import pyspark.sql.functions as sf
df.where(df.gengo == "平成").groupBy("kenmei")\
    .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")).show() 

+----------------------+--------------------+--------------------+
|                kenmei|            male_avg|          female_avg|
+----------------------+--------------------+--------------------+
|人口集中地区以外の地区|2.0976203166666668E7|2.2272045166666668E7|
|                佐賀県|            408192.5|            456442.5|
|                栃木県|   987741.8333333334|            999415.5|
|                京都府|  1268325.3333333333|  1360099.3333333333|
|                香川県|   485871.8333333333|   523763.6666666667|
|                愛媛県|   692188.3333333334|   774376.1666666666|
|                秋田県|   542928.3333333334|            604578.5|
|                広島県|           1387308.5|  1478006.8333333333|
|                宮崎県|            542386.5|            608793.0|
|              鹿児島県|            818506.0|            929134.0|
|                埼玉県|  3492880.3333333335|  3443447.8333333335|
|                三重県|            893167.5|   944959.6666666666|
|                島根県|            356034.

In [19]:
df_after_t = df.where(df.gengo == "平成").groupBy("kenmei")\
    .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("jinko_female_avg"))\
        .filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg")

In [20]:
df_after_t.show()

+--------+-----------------+-----------------+
|  kenmei|         male_avg| jinko_female_avg|
+--------+-----------------+-----------------+
|  鳥取県|287885.3333333333|314291.3333333333|
|  島根県|         356034.5|388621.6666666667|
|  高知県|372268.1666666667|         418517.0|
|  徳島県|383399.1666666667|         423152.0|
|  福井県|         395512.5|420182.6666666667|
|  佐賀県|         408192.5|         456442.5|
|  山梨県|425777.8333333333|441831.1666666667|
|  香川県|485871.8333333333|523763.6666666667|
|和歌山県|         490624.0|547112.3333333334|
|  富山県|         532857.0|573049.8333333334|
|  宮崎県|         542386.5|         608793.0|
|  秋田県|542928.3333333334|         604578.5|
|  石川県|         566064.0|         604518.5|
|  大分県|571530.6666666666|638773.6666666666|
|  山形県|         583603.5|627811.6666666666|
|  沖縄県|         654622.0|679050.6666666666|
|  岩手県|659592.6666666666|714973.1666666666|
|  滋賀県|         662391.0|         680326.0|
|  奈良県|671178.6666666666|734736.6666666666|
|  青森県|675238.6666666666

In [21]:
#カラムナーフォーマットへの変換
df_after_t.write.mode("overwrite").parquet("/Users/isomasaki/pyspark_batch/dataset/parquet")


In [22]:
!ls -l /Users/isomasaki/pyspark_batch/dataset/parquet

/Users/isomasaki/.zshenv:4: command not found: goenv
total 8
-rw-r--r--  1 isomasaki  staff     0 Jul 21 09:19 _SUCCESS
-rw-r--r--  1 isomasaki  staff  2083 Jul 21 09:19 part-00000-1f96165f-5412-4cc1-87e3-4200f6fe7bfb-c000.snappy.parquet


In [23]:
#スモールファイル問題に対処するためにファイルをまとめる(ノードの数)
df_after_t.repartition(1).write.mode("overwrite").parquet("/Users/isomasaki/pyspark_batch/dataset/parquet")

In [24]:
!ls -l /Users/isomasaki/pyspark_batch/dataset/parquet

/Users/isomasaki/.zshenv:4: command not found: goenv
total 8
-rw-r--r--  1 isomasaki  staff     0 Jul 21 09:19 _SUCCESS
-rw-r--r--  1 isomasaki  staff  2083 Jul 21 09:19 part-00000-e4f5624d-8fa8-4c24-b8ae-1f49d59da26f-c000.snappy.parquet


In [25]:
df_after_t.repartition(1).write.partitionBy("kenmei").mode("overwrite").parquet("/Users/isomasaki/pyspark_batch/dataset/parquet")

In [26]:
!ls -l /Users/isomasaki/pyspark_batch/dataset/parquet

/Users/isomasaki/.zshenv:4: command not found: goenv
total 0
-rw-r--r--  1 isomasaki  staff    0 Jul 21 09:19 _SUCCESS
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=三重県
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=京都府
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=人口集中地区
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=佐賀県
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=全国
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=兵庫県
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=北海道
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=千葉県
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=和歌山県
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=埼玉県
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=大分県
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=大阪府
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=奈良県
drwxr-xr-x  4 isomasaki  staff  128 Jul 21 09:19 kenmei=宮城県
drwxr-xr-x  4 isomasaki  staff  128 Ju

In [27]:
!ls -l /Users/isomasaki/pyspark_batch/dataset/parquet/kenmei=香川県

/Users/isomasaki/.zshenv:4: command not found: goenv
total 8
-rw-r--r--  1 isomasaki  staff  786 Jul 21 09:19 part-00000-af44ab2c-c3e8-4e15-b63d-6ed105633941.c000.snappy.parquet


In [28]:
#香川県のみのデータ
parquet_df = spark.read.parquet("/Users/isomasaki/pyspark_batch/dataset/parquet/kenmei=香川県")
parquet_df.show()

+-----------------+-----------------+
|         male_avg| jinko_female_avg|
+-----------------+-----------------+
|485871.8333333333|523763.6666666667|
+-----------------+-----------------+



In [29]:
spark.sql(
    """
    CREATE EXTERNAL TABLE IF NOT EXISTS default.jinko_avg (male_avg double,jinko_female_avg double)
    PARTITIONED BY (kenmei String)
    STORED AS PARQUET
    LOCATION '/Users/isomasaki/pyspark_batch/dataset/parquet/';
    """
)

DataFrame[]

In [30]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|jinko_avg|      false|
+---------+---------+-----------+



In [31]:
df_result = spark.sql("select * from default.jinko_avg")
df_result.show()
#partitionつきのテーブルでは、最初はテーブルの中身を確認できない

22/07/21 09:19:40 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
22/07/21 09:19:40 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
22/07/21 09:19:40 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/07/21 09:19:40 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
+--------------------+--------------------+------------+
|            male_avg|    jinko_female_avg|      kenmei|
+--------------------+--------------------+------------+
|            893167.5|   944959.6666666666|      三重県|
|  1268325.3333333333|  1360099.3333333333|      京都府|
|4.0840519833333336E7|4.2415789666666664E7|人口集中地区|
|         6.1816723E7|6.4687834833333336E7|        全国|
|           2650310.5|           2861527.0|      兵庫県|
|  2665781.3333333335|  2923371.8333333335|      北海道|
|           2987847.0|           2974638.5|      千葉県|


In [32]:
#メタデータにパーティションを登録
spark.sql("msck repair table jinko_avg")

DataFrame[]

In [33]:
df_result = spark.sql("select * from default.jinko_avg")
df_result.show()
#partitionつきのテーブルでは、最初はテーブルの中身を確認できない

+--------------------+--------------------+------------+
|            male_avg|    jinko_female_avg|      kenmei|
+--------------------+--------------------+------------+
|            893167.5|   944959.6666666666|      三重県|
|  1268325.3333333333|  1360099.3333333333|      京都府|
|4.0840519833333336E7|4.2415789666666664E7|人口集中地区|
|         6.1816723E7|6.4687834833333336E7|        全国|
|           2650310.5|           2861527.0|      兵庫県|
|  2665781.3333333335|  2923371.8333333335|      北海道|
|           2987847.0|           2974638.5|      千葉県|
|            490624.0|   547112.3333333334|    和歌山県|
|  3492880.3333333335|  3443447.8333333335|      埼玉県|
|   571530.6666666666|   638773.6666666666|      大分県|
|   4292675.833333333|           4517115.0|      大阪府|
|   671178.6666666666|   734736.6666666666|      奈良県|
|           1139561.5|           1191255.0|      宮城県|
|            542386.5|            608793.0|      宮崎県|
|            532857.0|   573049.8333333334|      富山県|
|   709497.8333333334|

In [34]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("chaper2")\
    .config("hive.exec.dynamic.partition","true")\
    .config("hive.exec.dynamic.partition.mode","nonstrict")\
    .config("spark.sql.session.timeZone","JST")\
    .config("spark.ui.enabled","true")\
    .enableHiveSupport()\
    .getOrCreate()
    

22/07/21 09:19:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [35]:
#仮想テーブルの作成
df.createOrReplaceTempView("jinko")

In [36]:
#sparkSQLの実行

spark.sql("select * from jinko").show()

+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|          code|    kenmei|gengo|    wareki|   seireki| chu|       sokei|jinko_male|jinko_female|
+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|都道府県コード|都道府県名| 元号|和暦（年）|西暦（年）|  注|人口（総数）|人口（男）|  人口（女）|
|            00|      全国| 大正|         9|      1920|null|    55963053|  28044185|    27918868|
|            01|    北海道| 大正|         9|      1920|null|     2359183|   1244322|     1114861|
|            02|    青森県| 大正|         9|      1920|null|      756454|    381293|      375161|
|            03|    岩手県| 大正|         9|      1920|null|      845540|    421069|      424471|
|            04|    宮城県| 大正|         9|      1920|null|      961768|    485309|      476459|
|            05|    秋田県| 大正|         9|      1920|null|      898537|    453682|      444855|
|            06|    山形県| 大正|         9|      1920|null|      968925|    478328|      49059

In [37]:
df_after_t_by_sql = spark.sql(
    """
    select 
      kenmei,
      avg(jinko_male) as male_avg,
      avg(jinko_female) as female_avg
    from 
      jinko
    where
      gengo = "平成"
      and kenmei != "人口集中地区以外の地区"
    group by kenmei
    order by male_avg
    """
)

df_after_t_by_sql.show()

+--------+-----------------+-----------------+
|  kenmei|         male_avg|       female_avg|
+--------+-----------------+-----------------+
|  鳥取県|287885.3333333333|314291.3333333333|
|  島根県|         356034.5|388621.6666666667|
|  高知県|372268.1666666667|         418517.0|
|  徳島県|383399.1666666667|         423152.0|
|  福井県|         395512.5|420182.6666666667|
|  佐賀県|         408192.5|         456442.5|
|  山梨県|425777.8333333333|441831.1666666667|
|  香川県|485871.8333333333|523763.6666666667|
|和歌山県|         490624.0|547112.3333333334|
|  富山県|         532857.0|573049.8333333334|
|  宮崎県|         542386.5|         608793.0|
|  秋田県|542928.3333333334|         604578.5|
|  石川県|         566064.0|         604518.5|
|  大分県|571530.6666666666|638773.6666666666|
|  山形県|         583603.5|627811.6666666666|
|  沖縄県|         654622.0|679050.6666666666|
|  岩手県|659592.6666666666|714973.1666666666|
|  滋賀県|         662391.0|         680326.0|
|  奈良県|671178.6666666666|734736.6666666666|
|  青森県|675238.6666666666

In [38]:
spark.sql(
    """
   insert overwrite table jinko_avg partition(kenmei)
   select 
      avg(jinko_male) as male_avg,
      avg(jinko_female),kenmei as female_avg
    from 
      jinko
    where
      gengo = "平成"
      and kenmei != "人口集中地区以外の地区"
    group by kenmei
    order by male_avg
    """
)

DataFrame[]

In [39]:
spark.sql("select * from jinko_avg").show()

+--------------------+--------------------+------------+
|            male_avg|    jinko_female_avg|      kenmei|
+--------------------+--------------------+------------+
|            893167.5|   944959.6666666666|      三重県|
|  1268325.3333333333|  1360099.3333333333|      京都府|
|4.0840519833333336E7|4.2415789666666664E7|人口集中地区|
|         6.1816723E7|6.4687834833333336E7|        全国|
|           2650310.5|           2861527.0|      兵庫県|
|  2665781.3333333335|  2923371.8333333335|      北海道|
|           2987847.0|           2974638.5|      千葉県|
|            490624.0|   547112.3333333334|    和歌山県|
|  3492880.3333333335|  3443447.8333333335|      埼玉県|
|   571530.6666666666|   638773.6666666666|      大分県|
|   4292675.833333333|           4517115.0|      大阪府|
|   671178.6666666666|   734736.6666666666|      奈良県|
|           1139561.5|           1191255.0|      宮城県|
|            542386.5|            608793.0|      宮崎県|
|            532857.0|   573049.8333333334|      富山県|
|   709497.8333333334|

In [40]:
#hint文でスモールファイル問題の解決 /** repartition(1) */ 忘れずに
#df2 でキャッシュ
df2 = spark.sql(
   """
   insert overwrite table jinko_avg partition(kenmei)
   select /** repartition(1) */
      avg(jinko_male) as male_avg,
      avg(jinko_female),kenmei as female_avg
    from 
      jinko
    where
      gengo = "平成"
      and kenmei != "人口集中地区以外の地区"
    group by kenmei
    order by male_avg
    """
)

In [41]:
#本番環境を想定したpyspark 昭和
spark.sql("select * from jinko_avg").show()

+--------------------+--------------------+------------+
|            male_avg|    jinko_female_avg|      kenmei|
+--------------------+--------------------+------------+
|            893167.5|   944959.6666666666|      三重県|
|  1268325.3333333333|  1360099.3333333333|      京都府|
|4.0840519833333336E7|4.2415789666666664E7|人口集中地区|
|         6.1816723E7|6.4687834833333336E7|        全国|
|           2650310.5|           2861527.0|      兵庫県|
|  2665781.3333333335|  2923371.8333333335|      北海道|
|           2987847.0|           2974638.5|      千葉県|
|            490624.0|   547112.3333333334|    和歌山県|
|  3492880.3333333335|  3443447.8333333335|      埼玉県|
|   571530.6666666666|   638773.6666666666|      大分県|
|   4292675.833333333|           4517115.0|      大阪府|
|   671178.6666666666|   734736.6666666666|      奈良県|
|           1139561.5|           1191255.0|      宮城県|
|            542386.5|            608793.0|      宮崎県|
|            532857.0|   573049.8333333334|      富山県|
|   709497.8333333334|

In [ ]:
#localhost:4040から

In [43]:
df2.cache()
df2.is_cached

22/07/21 09:36:12 WARN CacheManager: Asked to cache already cached data.


True

In [48]:
#テーブルをキャッシュ
spark.catalog.cacheTable("jinko_avg")
spark.catalog.isCached("jinko_avg")

True